In [1]:
from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from adp_core.agent import AgentMessage, BaseAgent
from adp_core.chain import BaseLLMChain
from adp_core.orchestration import ParallelAgent
from adp_llamaindex.chain import ChatCausalMultiTurnsChain
from llama_index.llms.ollama import Ollama

In [2]:
class Agent(BaseAgent):
    chain: BaseLLMChain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        self.state = "running"
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.card.name
        self.state = "idle"
        return message

In [3]:
system_prompt = """You are a genius and creative writer.
"""

user_prompt = """{query}"""

max_iteration = 5

def state_callback(state: str):
    print(f"agent state: {state}")
agent1_skill = AgentSkill(
    id='agent1-skill',
    name="agent1 skill",
    description="self-agent1 skill",
    tags=['agent1']
)
agent1_card = AgentCard(
    name="agent1 agent",
    description="self-agent1 agent",
    skills=[agent1_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
agent2_skill = AgentSkill(
    id='agent2-skill',
    name="agent2 skill",
    description="self-agent2 skill",
    tags=['agent2']
)
agent2_card = AgentCard(
    name="agent2 agent",
    description="self-agent2 agent",
    skills=[agent2_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
parallel_skill = AgentSkill(
    id='parallel-skill',
    name="parallel skill",
    description="self-parallel skill",
    tags=['parallel']
)
parallel_card = AgentCard(
    name="parallel agent",
    description="self-parallel agent",
    skills=[parallel_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)

model = Ollama(model="llama3.2:latest", base_url="192.168.55.1:11434", request_timeout=400, context_window=16384)
chain = ChatCausalMultiTurnsChain(model=model, system_prompt=system_prompt, user_prompt_template=user_prompt, base_url="192.168.55.1:11434", request_timeout=300, context_window=32768)
agent1 = Agent(chain=chain, card=agent1_card, state_change_callback=state_callback)
agent2 = Agent(chain=chain, card=agent2_card, state_change_callback=state_callback)
parallel_agent = ParallelAgent(agents=[agent1, agent2], card=parallel_card, state_change_callback=state_callback)

In [4]:
query = "Write a short poem with Shakespere's style about love"
message = parallel_agent.execute(AgentMessage(query=query), temperature=1.5)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: parallel agent:running/((agent1 agent:idle)|(agent2 agent:idle))
agent state: parallel agent:running/((agent1 agent:running)|(agent2 agent:idle))
agent state: parallel agent:running/((agent1 agent:running)|(agent2 agent:running))
agent state: parallel agent:running/((agent1 agent:idle)|(agent2 agent:running))
agent state: parallel agent:running/((agent1 agent:idle)|(agent2 agent:idle))
agent state: parallel agent:idle/((agent1 agent:idle)|(agent2 agent:idle))
query='' query_media=None query_media_type=None origin='parallel agent' responses=[('agent2 agent', "O, fairest love, thou dost entwine my heart\nWith tender threads of sweetest, secret art\nAs morning dew doth glisten on the grass,\nThy beauty shines, and doth forever last.\n\nThy eyes, like sapphires, shining bright and blue,\nDo sparkle with a fire that doth anew\nMy passions, erst dormant, now take flight,\nAnd in thine arms, I find my guiding light.\n\nO, love, thou art the master of my soul,\nThe whispered promi